# Stochastic Gradient Descent (SGD) on Logistic Regression

Author: Alexandre Gramfort

### Table of content

[1. Loss functions, gradients and step-sizes](#loss)<br>
[2. Generate a dataset](#data)<br>
[3. From GD to SGD](#sgd)<br>

In [ ]:
%matplotlib inline

In [ ]:
from time import time

import numpy as np
from scipy.linalg import norm
import matplotlib.pyplot as plt
from numba import njit

<a id='loss'></a>
## Loss functions, gradients and step-sizes

We want to minimize
$$
f(w) = \frac 1n \sum_{i=1}^n \ell(x_i^\top w, y_i) + \frac \lambda 2 \|w\|_2^2
$$
where $\ell(z, y) = \log(1 + \exp(-yz))$ (logistic regression).

We write it as a minimization problem of the form
$$
f(w) = \frac 1n \sum_{i=1}^n f_i(w)
$$
where
$$
f_i(w) = \ell(x_i^\top w, y_i) + \frac \lambda 2 \|w\|_2^2.
$$

The gradient of f_i reads:
$$
\nabla f_i(w) = - \frac{y_i}{1 + \exp(y_i x_i^\top w)} x_i + \lambda w.
$$

We now introduce functions that will be used for the solvers.


In [ ]:
@njit    
def f_grad_i(i, w, X, y, lambd):
    """Gradient with respect to a sample"""
    x_i = X[i]
    y_i = y[i]
    return - x_i * y_i / (1. + np.exp(y_i * np.dot(x_i, w))) + lambd * w


@njit
def f_grad(w, X, y, lambd):
    """Full gradient"""
    g = np.zeros_like(w)
    n_samples = X.shape[0]
    for i in range(n_samples):
        g += f_grad_i(i, w, X, y, lambd)
    return g / n_samples


def f(w, X, y, lambd):
    yXw = y * np.dot(X, w)
    return np.mean(np.log(1. + np.exp(- yXw))) + lambd * norm(w) ** 2 / 2.

<a id='data'></a>
## 2. Generate a dataset

We generate datasets for the logistic regression model.

In [ ]:
from numpy.random import multivariate_normal, randn
from scipy.linalg.special_matrices import toeplitz


def simulate(w, n_samples, std=1., corr=0.5):
    """Simulation for the logistic regression problem.
    
    Parameters
    ----------
    w : ndarray, shape (d,)
        The coefficients of the model
    n_samples : int
        Sample size    
    std : float, default=1.
        Standard-deviation of the noise
    corr : float, default=0.5
        Correlation of the features matrix
    
    Returns
    -------
    X : ndarray, shape (n_samples, n_features)
        The design matrix.
    y : ndarray, shape (n_samples,)
        The targets.
    """    
    n_features = w.shape[0]
    cov = toeplitz(corr ** np.arange(0, n_features))
    X = multivariate_normal(np.zeros(n_features), cov, size=n_samples)
    noise = std * randn(n_samples)
    return X, np.sign(X @ w + noise)

In [ ]:
n_features = 50
n_samples = 10000
idx = np.arange(n_features)

# Ground truth coefficients of the model
w_true = (-1)**idx * np.exp(-idx / 10.)

simulate(w_true, n_samples, std=1., corr=0.7);

In [ ]:
plt.stem(w_true);

### Numerically check loss and gradient

In [ ]:
from scipy.optimize import check_grad

lambd = 1. / n_samples ** (0.5)

X, y = simulate(w_true, n_samples, std=1., corr=0.1)

# Check that the gradient and the loss numerically match
check_grad(f, f_grad, np.random.randn(n_features), X, y, lambd)

### Get a very precise minimum to compute distances to minimum

In [ ]:
from scipy.optimize import fmin_l_bfgs_b

w_init = np.zeros(n_features)
w_min, f_min, _ = fmin_l_bfgs_b(f, w_init, f_grad,
                                args=(X, y, lambd), pgtol=1e-30, factr=1e-30)

print(f_min)
print(norm(f_grad(w_min, X, y, lambd)))

<a id='sgd'></a> 

## 3. From Gradient Descent (GD) to Stochastic Gradient Descent (SGD)

### Define a class to monitor iterations

In [ ]:
class monitor:
    def __init__(self, algo, f, w_min, args=()):
        self.w_min = w_min
        self.algo = algo
        self.f = f
        self.args = args
        self.f_min = f(w_min, *args)
    
    def run(self, *algo_args, **algo_kwargs):
        t0 = time()
        _, w_list = self.algo(*algo_args, **algo_kwargs)
        self.total_time = time() - t0
        self.w_list = w_list
        self.err = [norm(w - self.w_min) for w in w_list]
        self.obj = [self.f(w, *self.args) - self.f_min for w in w_list]

In [ ]:
# Number of iterations
n_iter = 50

In [ ]:
@njit
def gd(w_init, grad, n_iter=100, step=1., args=()):
    """Gradient descent algorithm."""
    w = w_init.copy()
    w_list = []
    for i in range(n_iter):
        w -= step * grad(w, *args)
        w_list.append(w.copy())
    return w, w_list

In [ ]:
def lipschitz_logreg(X, y, lambd):
    return norm(X, ord=2) ** 2 / (4. * len(X)) + lambd

step = 1. / lipschitz_logreg(X, y, lambd)
w_init = np.zeros(n_features)
monitor_gd = monitor(gd, f, w_min, (X, y, lambd))
monitor_gd.run(w_init, f_grad, n_iter, step, args=(X, y, lambd))

### A first numerical comparison of a deterministic gradient descent

First, define some plotting functions.

In [ ]:
def plot_epochs(monitors, solvers):
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 2, 1)
    for monit in monitors:
        plt.semilogy(monit.obj, lw=2)
        plt.xlabel("Epoch")
        plt.ylabel("$f(w_k) - f(w^*)$")

    plt.legend(solvers)

    plt.subplot(1, 2, 2)

    for monit in monitors:
        objs = monit.obj
        plt.semilogy(np.linspace(0, monit.total_time, len(objs)), objs, lw=2)
        plt.xlabel("Timing")
        plt.ylabel("$f(w_k) - f(w^*)$")

    plt.legend(solvers)

In [ ]:
monitors = [monitor_gd]
solvers = ["GD"]
plot_epochs(monitors, solvers)

<a id='stoc'></a> 
## 4. Stochastic methods

In [ ]:
n_iter = 50

# generate indices of random samples
iis = np.random.randint(0, n_samples, n_samples * n_iter)

### SGD

We recall that an iteration of SGD writes

- Pick $i$ uniformly at random in $\{1, \ldots, n\}$
- Apply
$$
w_{t+1} \gets w_t - \frac{\eta_0}{\sqrt{t+1}} \nabla f_i(w_t)
$$

where $\eta_0$ is a step-size to be tuned by hand.

In [ ]:
@njit
def sgd(w_init, iis, grad_i, n_iter=100, step=1., store_every=n_samples, args=()):
    """Stochastic gradient descent algorithm."""
    w = w_init.copy()
    w_list = []
    for idx in range(n_iter):
        i = iis[idx]

        w -= step / np.sqrt(idx + 1) * grad_i(i, w, *args) 

        # Update metrics after each iteration.
        if idx % store_every == 0:
            w_list.append(w.copy())
    return w, w_list

In [ ]:
step0 = 1e-1
w_init = np.zeros(n_features)

monitor_sgd = monitor(sgd, f, w_min, (X, y, lambd))
monitor_sgd.run(w_init, iis, f_grad_i, n_iter * n_samples, step0, args=(X, y, lambd))

In [ ]:
monitors = [monitor_gd, monitor_sgd]
solvers = ["GD", "SGD"]
plot_epochs(monitors, solvers)

<div class="alert alert-success">
    <b>QUESTIONS:</b>
     <ul>
       <li>Change the value of n_samples and show that SGD becomes even more competitive when n_samples is large.</li>
       <li>Change the regularization (the ``lambd`` parameter) to low regularization $\lambda = 1 / \textrm{n_samples}$ and high regularization $\lambda = 1 / \sqrt{\textrm{n_samples}}$ and compare your results.</li>
       <li>Change the correlation parameter (corr parameter). Show that a high correlation slows down convergencee.</li>
    </ul>
</div>